<h1 style="padding: 25px 25px; background-color: #F5B041; font-family: Sans-Serif; color:black; text-align: center">
Project: Safety Helmet Detection</h1>

<h2 style="padding: 12px 12px; background-color: #F5B041; font-family: Sans-Serif; color:black">
Overview</h2>

Goal of the project is to detect the presence of people and safety helmets thus improving work safety. 

<h2 style="padding: 12px 12px; background-color: #F5B041; font-family: Sans-Serif; color:black">
Table of contents</h2>

1. [Data Description](#data-description) 
2. [Import modules](#import-modules)
3. [Configuration](#configuration)
4. [Data Exploration](#data-exploration)
5. [Train](#train)
6. [Test](#test)

<a id="data-description"></a>
<h2 style="padding: 12px 12px; background-color: #F5B041; font-family: Sans-Serif; color:black">
Data description</h2>

- The dataset is a great collection of images, since the labels are in the following format: 'Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone', 'Safety Vest', 'machinery', 'vehicle'

- **Quick Summary**
    - Number of classes: 10
    - Label Annotation: YOLO format (.txt)
    - Metadata: metadata.csv and count.csv provides information about the dataset and train-val-test count information.
    - PPE Class Map: {0: 'Hardhat', 1: 'Mask', 2: 'NO-Hardhat', 3: 'NO-Mask', 4: 'NO-Safety Vest', 5: 'Person', 6: 'Safety Cone', 7: 'Safety Vest', 8: 'machinery', 9: 'vehicle'}
    - Difficulty: This is a beginner-friendly dataset on multi-class classification, object detection, and tracking. Annotations are in YoloV8 format. The splits are given in the dataset folder itself with metadata, so anyone can use this data to run models and produce results.


<a id="import-modules"></a>
<h2 style="padding: 12px 12px; background-color: #F5B041; font-family: Sans-Serif; color:black">
Import modules</h2>

In [1]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from IPython.core.magic import register_line_cell_magic
import yaml
from PIL import Image
import os
import seaborn as sns
from ultralytics import YOLO
from ultralytics import NAS
from matplotlib.patches import Rectangle
import glob
import cv2


<a id="configuration"></a>
<h2 style="padding: 12px 12px; background-color: #F5B041; font-family: Sans-Serif; color:black">
Configuration</h2>

In [3]:
warnings.filterwarnings("ignore")

np.random.seed(12345)

%matplotlib inline

!wandb disabled

W&B disabled.


In [5]:
INPUT_DIR = '/kaggle/input/construction-site-safety-image-dataset-roboflow/css-data'
WORK_DIR = '/kaggle/working/'

In [4]:
INPUT_DIR = '/kaggle/input/godsensi/data'
WORK_DIR = '/kaggle/working/'

<a id="Data Exploration"></a>
<h2 style="padding: 12px 12px; background-color: #F5B041; font-family: Sans-Serif; color:black">
Data Exploration</h2>

<h4 style="border-bottom: 3px solid #FAD7A0; padding: 12px 12px; font-family: Sans-Serif; color:black">
<b>Create data YAML file</b></h4>

In [6]:
num_classes = 10
classes = ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone', 'Safety Vest', 'machinery', 'vehicle']

In [7]:
dict_file = {'train': os.path.join(INPUT_DIR, 'train'),
             'val': os.path.join(INPUT_DIR, 'valid'),
             'test': os.path.join(INPUT_DIR, 'test'),
             'nc': num_classes,
             'names': classes
            }

with open(os.path.join(WORK_DIR, 'data.yaml'), 'w+') as file:
    yaml.dump(dict_file, file)

<h4 style="border-bottom: 3px solid #FAD7A0; padding: 12px 12px; font-family: Sans-Serif; color:black">
<b>Check class balance</b></h4>

In [ ]:
class_idx = {str(i):classes[i] for i in range(num_classes)}

class_stat = {}
data_len = {}

for mode in ['train', 'valid', 'test']:

    class_count = {classes[i]:0 for i in range(num_classes)}

    path = os.path.join(INPUT_DIR, mode, 'labels')

    for file in os.listdir(path):

        with open(os.path.join(path, file)) as f:
            lines = f.readlines()

            for cls in set([line[0] for line in lines]):
                class_count[class_idx[cls]] += 1
                
    data_len[mode] = len(os.listdir(path))
    class_stat[mode] = class_count

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(15, 5), sharey=True)

for i, mode in enumerate(['train', 'valid', 'test']):
    sns.barplot(pd.DataFrame({mode:class_stat[mode]}).T/data_len[mode]*100, ax=ax[i])
    ax[i].set_title(mode)
    ax[i].tick_params(rotation=90)
    ax[i].set_ylabel('Percenatage of classes')
plt.show()

- There is imbalance in classes


- Overall class distribution between train, validation and test sets are similar, although validation and test sets have significantly lower percentage of `machinery` class

<h4 style="border-bottom: 3px solid #FAD7A0; padding: 12px 12px; font-family: Sans-Serif; color:black">
<b>Check image sizes</b></h4>

In [ ]:
for mode in ['train', 'valid', 'test']:
    print(f'\nImage sizes in {mode} set:\n')
    img_size = 0
    for file in glob.glob(os.path.join(INPUT_DIR, mode, 'images', '*')):
        image = Image.open(file)
        if image.size != img_size:
            print(f'\t{image.size}')
            img_size = image.size

- All images have size `640x640`

<h4 style="border-bottom: 3px solid #FAD7A0; padding: 12px 12px; font-family: Sans-Serif; color:black">
<b>Check dataset sizes</b></h4>

In [ ]:
for mode in ['train', 'valid', 'test']:
    
    files =  glob.glob(os.path.join(INPUT_DIR, mode, 'images', '*'))
        
    print(f'{mode} set size: {len(files)}\n')

<a id="train"></a>

<h2 style="padding: 12px 12px; background-color: #F5B041; font-family: Sans-Serif; color:black">
Train</h2>

<h4 style="border-bottom: 3px solid #FAD7A0; padding: 12px 12px; font-family: Sans-Serif; color:black">
<b>Arguments for training</b></h4>

- **task**: preform `detect`, `segment` or `classify`
- **imgsz**: define input image size
- **batch**: determine batch size
- **epochs**: define the number of training epochs. (Note: often, 3000+ are common here!)
- **data**: set the path to our yaml file
- **mode**: mode - `train`, `val` or `predict`
- **model**: model to use (could be pre-trained)
- **name**: result names

<h4 style="border-bottom: 3px solid #FAD7A0; padding: 12px 12px; font-family: Sans-Serif; color:black">
<b>Load model<b></h4>

- Load `pre-trained` YOLO-nano model 

In [ ]:
import torch

# Load the first YOLO model
model1 = YOLO('yolov8n.pt')

# Load the second YOLO model
model2 = YOLO('yolov8n.pt')


In [8]:
model = YOLO('yolov8n.pt')

<h4 style="border-bottom: 3px solid #FAD7A0; padding: 12px 12px; font-family: Sans-Serif; color:black">
<b>Train</b></h4>

In [9]:
model.train(data=os.path.join(WORK_DIR,'data.yaml'),
            task='detect',
            imgsz=640,
            epochs=30,
            batch=32,
            mode='train',
            name='yolov8n_v1_train')

New https://pypi.org/project/ultralytics/8.1.15 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.145 🚀 Python-3.7.12 torch-1.13.0 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/kaggle/working/data.yaml, epochs=30, patience=50, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_v1_train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_w

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

<a id="test"></a>

<h2 style="padding: 12px 12px; background-color: #F5B041; font-family: Sans-Serif; color:black">
Test</h2>

<h4 style="border-bottom: 3px solid #FAD7A0; padding: 12px 12px; font-family: Sans-Serif; color:black">
<b>Select best model</b></h4>

In [14]:
model = YOLO('/kaggle/working/runs/detect/yolov8n_v1_train/weights/best.pt') 
#/kaggle/input/construction-site-safety-image-dataset-roboflow/results_yolov8n_100e/kaggle/working/runs/detect/train/weights/last.pt
#model= YOLO('/kaggle/working/runs/detect/yolov8n_v1_train2/weights/last.pt')


In [13]:
num_classes = 10
classes = [' Hardhat', ' Mask', ' NO-Hardhat', ' NO-Mask', ' NO-Safety Vest', ' Person', ' Safety Cone', ' Safety Vest', ' machinery', ' vehicle']

In [15]:
submission=pd.read_csv("/kaggle/input/godsensi/SampleSubmission.csv")

In [16]:
test_csv=pd.read_csv("/kaggle/input/godsensi/Test.csv")

In [17]:
submission['filename']=test_csv['filename']

In [26]:
probability

0.0

In [25]:
submission.to_csv("mm.csv",index=False)

In [18]:
import torch

for index, row in submission.iterrows():
    results = model.predict(source=os.path.join(INPUT_DIR, 'test', row['filename']), save=True)
    classes = ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone', 'Safety Vest', 'machinery', 'vehicle']
    class_probabilities = {class_name: 0.0 for class_name in classes}
    print(class_probabilities)
    for result in results:
        if result.boxes:
            for box in result.boxes:
                class_id = int(box.cls)
                class_name = model.names[class_id]
                class_probabilities[class_name] = max(class_probabilities[class_name], box.conf)
    print('Waywaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa')
    print(class_probabilities)
    
    for class_name, probability in class_probabilities.items():
        submission.loc[index, " "+class_name] = probability.item() if isinstance(probability, torch.Tensor) else probability

submission.fillna(0, inplace=True)


image 1/1 /kaggle/input/godsensi/data/test/006463_jpg.rf.02f19082420ecc5537b9d59abbe6050c.jpg: 640x640 9 Hardhats, 6 NO-Masks, 8 NO-Safety Vests, 11 Persons, 1 Safety Vest, 8.0ms
Speed: 1.5ms preprocess, 8.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/youtube-34_jpg.rf.03eacc444bae3c5caa3fef5c736c3e40.jpg: 640x640 2 Persons, 1 machinery, 1 vehicle, 8.8ms
Speed: 1.5ms preprocess, 8.8ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/IMG_0871_mp4-23_jpg.rf.03f872b1ed87ad7fadc85e09475ad37a.jpg: 640x640 1 Mask, 1 NO-Hardhat, 1 NO-Safety Vest, 1 Person, 8.1ms
Speed: 1.4ms preprocess, 8.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/youtube-596_jpg.rf.11a8a4ac01d8aadb80eeb0406dfa579a.jpg: 64

{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': 0.0}
Waywaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
{'Hardhat': tensor([0.9234], device='cuda:0'), 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': tensor([0.8040], device='cuda:0'), 'NO-Safety Vest': tensor([0.9104], device='cuda:0'), 'Person': tensor([0.9134], device='cuda:0'), 'Safety Cone': 0.0, 'Safety Vest': tensor([0.4170], device='cuda:0'), 'machinery': 0.0, 'vehicle': 0.0}
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': 0.0}
Waywaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': tensor([0.3274], device='cuda:0'), 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': tensor([0.6390], device='cuda:0'), 'vehicle': tensor([0


image 1/1 /kaggle/input/godsensi/data/test/2008_008519_jpg.rf.1798c8eed7de04399a0e7e297b4b4c9e.jpg: 640x640 2 Hardhats, 2 NO-Safety Vests, 6 Persons, 2 Safety Vests, 1 machinery, 8.3ms
Speed: 1.3ms preprocess, 8.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/youtube-388_jpg.rf.18caa1da4f818a65f73e48463cb2270e.jpg: 640x640 3 machinerys, 8.7ms
Speed: 1.3ms preprocess, 8.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/004063_jpg.rf.1b7cdc4035bcb24ef69b8798b444053e.jpg: 640x640 5 Hardhats, 5 NO-Safety Vests, 6 Persons, 1 Safety Vest, 8.2ms
Speed: 1.3ms preprocess, 8.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/youtube-186_jpg.rf.1f1d93447d4be3233c22c4ce9f6e0601.jpg: 640x640 1 Hardhat,

Waywaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': tensor([0.4081], device='cuda:0'), 'Safety Cone': tensor([0.7346], device='cuda:0'), 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': tensor([0.9523], device='cuda:0')}
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': 0.0}
Waywaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
{'Hardhat': tensor([0.3929], device='cuda:0'), 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': tensor([0.7693], device='cuda:0'), 'Person': tensor([0.7171], device='cuda:0'), 'Safety Cone': 0.0, 'Safety Vest': tensor([0.6725], device='cuda:0'), 'machinery': tensor([0.5572], device='cuda:0'), 'vehicle': 0.0}
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machine

image 1/1 /kaggle/input/godsensi/data/test/image_55_jpg.rf.27ae4341a9b9647d73a8929ff7a22369.jpg: 640x640 1 Hardhat, 1 Mask, 1 Person, 1 Safety Vest, 8.6ms
Speed: 1.4ms preprocess, 8.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/IMG_5846_jpg.rf.0b9984069116e2acd928568eb7a8e214.jpg: 640x640 21 Hardhats, 30 NO-Masks, 38 Persons, 24 Safety Vests, 9.2ms
Speed: 1.4ms preprocess, 9.2ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/bookstore_44_08_flickr_jpg.rf.244b6f308c529933af798c4063e58601.jpg: 640x640 (no detections), 8.7ms
Speed: 1.4ms preprocess, 8.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/0_jpg.rf.2ff49f74309118f169e07aa12564df87.jpg: 640x640 1 Hardhat, 8 Persons, 1 Safety Vest, 

...

{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': 0.0}
Waywaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
{'Hardhat': 0.0, 'Mask': tensor([0.8889], device='cuda:0'), 'NO-Hardhat': tensor([0.5099], device='cuda:0'), 'NO-Mask': 0.0, 'NO-Safety Vest': tensor([0.3334], device='cuda:0'), 'Person': tensor([0.9291], device='cuda:0'), 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': 0.0}
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': 0.0}
Waywaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': tensor([0.7623], device='cuda:0'), 'vehicle': 0.0}
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0,

image 1/1 /kaggle/input/godsensi/data/test/-3154-_png_jpg.rf.f118da2b1c20afb78ff93dc7d558f42c.jpg: 640x640 1 Hardhat, 2 NO-Masks, 1 NO-Safety Vest, 1 Person, 8.4ms
Speed: 1.3ms preprocess, 8.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/autox3_mp4-78_jpg.rf.dc5c00104c4cf733c2c06c820b82d338.jpg: 640x640 9 Safety Cones, 1 vehicle, 8.0ms
Speed: 1.4ms preprocess, 8.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/-211-_png_jpg.rf.eac228e3993f9027795b6400262811e7.jpg: 640x640 2 Hardhats, 3 NO-Masks, 1 Person, 2 Safety Vests, 8.6ms
Speed: 1.4ms preprocess, 8.6ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/youtube-226_jpg.rf.e6dcacf16dc2dbcfd29f6705b7c1a224.jpg: 640x640 1 Hardhat, 1 Person, 

{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': 0.0}
Waywaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
{'Hardhat': tensor([0.9201], device='cuda:0'), 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': tensor([0.2922], device='cuda:0'), 'NO-Safety Vest': tensor([0.8299], device='cuda:0'), 'Person': tensor([0.8070], device='cuda:0'), 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': 0.0}
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': 0.0}
Waywaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': tensor([0.8757], device='cuda:0'), 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': tensor([0.8139], device='cuda:0')}
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-

image 1/1 /kaggle/input/godsensi/data/test/youtube-367_jpg.rf.fab8b016cb63c0928f40849db44ee98d.jpg: 640x640 3 machinerys, 8.7ms
Speed: 1.4ms preprocess, 8.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/youtube-232_jpg.rf.f92f310106a11e01495ce8bfae0f9b5a.jpg: 640x640 1 Person, 8.2ms
Speed: 1.3ms preprocess, 8.2ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/construction-1027-_jpg.rf.e78f55761814a8726ea51993407957af.jpg: 640x640 2 machinerys, 8.4ms
Speed: 1.4ms preprocess, 8.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/detect/predict

image 1/1 /kaggle/input/godsensi/data/test/IMG_3100_mp4-3_jpg.rf.f05ef399880b1ac30e900df4f2fbb3c0.jpg: 640x640 1 Hardhat, 1 Mask, 1 NO-Hardhat, 1 NO-Safety Vest, 1 Person, 8.7ms
Speed: 1.3ms preprocess, 8.7ms in

{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': 0.0}
Waywaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': tensor([0.9056], device='cuda:0'), 'vehicle': 0.0}
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': 0.0}
Waywaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': tensor([0.4806], device='cuda:0'), 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery': 0.0, 'vehicle': 0.0}
{'Hardhat': 0.0, 'Mask': 0.0, 'NO-Hardhat': 0.0, 'NO-Mask': 0.0, 'NO-Safety Vest': 0.0, 'Person': 0.0, 'Safety Cone': 0.0, 'Safety Vest': 0.0, 'machinery'

In [19]:
submission

,ID,Hardhat,Mask,NO-Hardhat,NO-Mask,NO-Safety Vest,Person,Safety Cone,Safety Vest,machinery,vehicle,filename
0,ID_1,0.923418,0.000000,0.000000,0.803961,0.910405,0.913449,0.000000,0.417002,0.000000,0.000000,006463_jpg.rf.02f19082420ecc5537b9d59abbe6050c...
1,ID_2,0.000000,0.000000,0.000000,0.000000,0.000000,0.327397,0.000000,0.000000,0.639019,0.861964,youtube-34_jpg.rf.03eacc444bae3c5caa3fef5c736c...
2,ID_3,0.000000,0.896389,0.709123,0.000000,0.796589,0.917995,0.000000,0.000000,0.000000,0.000000,IMG_0871_mp4-23_jpg.rf.03f872b1ed87ad7fadc85e0...
3,ID_4,0.000000,0.000000,0.000000,0.000000,0.316987,0.940843,0.000000,0.000000,0.000000,0.000000,youtube-596_jpg.rf.11a8a4ac01d8aadb80eeb0406df...
4,ID_5,0.000000,0.000000,0.000000,0.000000,0.000000,0.408148,0.734647,0.000000,0.000000,0.952296,ka_01181_png_jpg.rf.154ee4ef254eabd62e316be504...
...,...,...,...,...,...,...,...,...,...,...,...,...
191,ID_192,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.905594,0.000000,youtube-367_jpg.rf.fab8b016cb63c0928f40849db44...
192,ID_193,0.000000,0.000000,0.000000,0.000000,0.000000,0.480590,0.000000,0.000000,0.000000,0.000000,youtube-232_jpg.rf.f92f310106a11e01495ce8bfae0...
193,ID_194,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.607490,0.000000,construction-1027-_jpg.rf.e78f55761814a8726ea5...
194,ID_195,0.331814,0.914295,0.535347,0.000000,0.716660,0.936713,0.000000,0.000000,0.000000,0.000000,IMG_3100_mp4-3_jpg.rf.f05ef399880b1ac30e900df4...


In [ ]:
for index, row in submission.iterrows():
    results = model.predict(conf=0.3,source=os.path.join(INPUT_DIR, 'test', row['filename']), save=True)
    object_names = []
    for result in results:
        if result.boxes:
            for i in result.boxes.cls:
                class_id = int(i)
                object_name = model.names[class_id]
                object_names.append(object_name)
        print('Waywaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa')
        
        object_names = list(set(object_names))
        print(object_names)
    for class_name in object_names:
        submission.loc[index, " "+class_name] = 1

submission.fillna(0, inplace=True)
print(submission.head())

In [ ]:
submission

In [ ]:
submission.columns

In [20]:
submission.drop(['filename'],axis=1,inplace=True)

In [21]:
submission.to_csv('submissionnyoo.csv', index=False)

In [ ]:
# Define the columns you want to keep
columns_to_keep = ['ID']+classes

# Create a new DataFrame with only the specified columns
extracted_submission = submission[columns_to_keep]

# Write the DataFrame to a CSV file
extracted_submission.to_csv('SubmissionBest14.csv', index=False)

### yoyo

In [ ]:
d = {0: ' Hardhat',
 1: ' Mask',
 2: ' NO-Hardhat',
 3: ' NO-Mask',
 4: ' NO-Safety Vest',
 5: ' Person',
 6: ' Safety Cone',
 7: ' Safety Vest',
 8: ' machinery',
 9: ' vehicle'}

In [ ]:
test = pd.read_csv('/kaggle/input/godsensi/Test.csv')
test.columns

In [ ]:
ss=pd.read_csv("/kaggle/input/godsensi/SampleSubmission.csv")
ss.columns

In [ ]:
test_images = test['filename'].tolist()
test_ids = test['ID'].tolist()

In [ ]:
predictions = model.predict(test_images)  # Assuming test_images is a list of image paths

for res in predictions:
    probabilities = res[0].scores.tolist()  # Assuming res[0].scores contains class probabilities
    classes = [d[i] for i in res[0].boxes.cls.tolist()]  # Assuming d is the class mapping dictionary
    # Initialize all probabilities to 0
    prob_dict = {class_name: 0 for class_name in ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone', 'Safety Vest', 'machinery', 'vehicle']}
    
    # Update probabilities for detected classes
    for prob, class_name in zip(probabilities, classes):
        prob_dict[class_name] = prob
    
    # Append the probabilities to respective lists
    Hardhat.append(prob_dict['Hardhat'])
    Mask.append(prob_dict['Mask'])
    NO_Hardhat.append(prob_dict['NO-Hardhat'])
    NO_Mask.append(prob_dict['NO-Mask'])
    NO_Safety_Vest.append(prob_dict['NO-Safety Vest'])
    Person.append(prob_dict['Person'])
    Safety_Cone.append(prob_dict['Safety Cone'])
    Safety_Vest.append(prob_dict['Safety Vest'])
    machinery.append(prob_dict['machinery'])
    vehicle.append(prob_dict['vehicle'])


In [ ]:
Hardhat = []
Mask = []
NO_Hardhat = []
NO_Mask = []
NO_Safety_Vest = [] 
Person = []
Safety_Cone = []
Safety_Vest = []
machinery = []
vehicle = []
for image in test_images:
    res = model.predict(f"/kaggle/input/godsensi/data/test/{image}")
    #res= model.predict(source=os.path.join(INPUT_DIR, 'test'), save=True)
    l  = list(set([int(i) for i in res[0].boxes.cls.tolist()]))
    response = [d[i] for i in l]
    if ' Hardhat' in response:
        Hardhat.append(1)
    else:
        Hardhat.append(0)
    if ' Mask' in response:
        Mask.append(1)
    else:
        Mask.append(0)
    if ' NO-Hardhat' in response:
        NO_Hardhat.append(1)
    else:
        NO_Hardhat.append(0)
    if ' NO-Mask' in response:
        NO_Mask.append(1)
    else:
        NO_Mask.append(0)
    if ' NO-Safety Vest' in response:
        NO_Safety_Vest.append(1)
    else:
        NO_Safety_Vest.append(0)
    if ' Person' in response:
        Person.append(1)
    else:
        Person.append(0)
    if ' Safety Cone' in response:
        Safety_Cone.append(1)
    else:
        Safety_Cone.append(0)
    if ' Safety Vest' in response:
        Safety_Vest.append(1)
    else:
        Safety_Vest.append(0)
    if ' machinery' in response:
        machinery.append(1)
    else:
        machinery.append(0)
    if 'vehicle' in response:
        vehicle.append(1)
    else:
        vehicle.append(0)

In [ ]:
ss=pd.read_csv("/kaggle/input/godsensi/SampleSubmission.csv")
ss.columns

In [ ]:
submission = pd.DataFrame({'ID': test_ids, ' Hardhat': Hardhat, ' Mask': Mask, ' NO-Hardhat': NO_Hardhat, ' NO-Mask': NO_Mask, ' NO-Safety Vest': NO_Safety_Vest, ' Person': Person, ' Safety Cone': Safety_Cone, ' Safety Vest': Safety_Vest, ' machinery': machinery, ' vehicle': vehicle})

In [ ]:
submission

In [ ]:
submission.to_csv('submissionnn-43.csv', index=False)

<h4 style="border-bottom: 3px solid #FAD7A0; padding: 12px 12px; font-family: Sans-Serif; color:black">
<b>Make predicitons on test set</b></h4>

In [ ]:
results = model.predict(source=os.path.join(INPUT_DIR, 'test', 'images'), save=True)

In [ ]:
results

<h4 style="border-bottom: 3px solid #FAD7A0; padding: 12px 12px; font-family: Sans-Serif; color:black">
<b>Show prediction results</b></h4>

In [ ]:
%matplotlib inline

predicitions = glob.glob(os.path.join(WORK_DIR, 'runs/detect/predict2', '*'))

n = 10

for i in range(n):
    idx = np.random.randint(0, len(predicitions))
    image = Image.open(predicitions[idx])
    plt.imshow(image)
    plt.grid(False)
    plt.show()